In [ ]:
img_cus = tf.image.rgb_to_grayscale(test_image)
img_final = np.expand_dims(np.array(img_cus),axis=3)

In [ ]:
!pip install kaggle

In [ ]:
from google.colab import files
files.upload()

In [ ]:
!mkdir -p ~/.kaggle 
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json 

In [ ]:
!kaggle datasets download -d datamunge/sign-language-mnist

sign-language-mnist.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
!unzip sign-language-mnist.zip

Archive:  sign-language-mnist.zip
replace amer_sign2.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace amer_sign3.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace american_sign_language.PNG? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace sign_mnist_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace sign_mnist_test/sign_mnist_test.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace sign_mnist_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace sign_mnist_train/sign_mnist_train.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
!ls

 american_sign_language.PNG   kaggle.json	        sign_mnist_test.csv
 amer_sign2.png		      sample_data	        sign_mnist_train
 amer_sign3.png		      sign-language-mnist.zip   sign_mnist_train.csv
'kaggle (1).json'	      sign_mnist_test


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv('/content/sign_mnist_train.csv')

In [ ]:
df_test = pd.read_csv('/content/sign_mnist_test.csv')

In [ ]:
df_test.shape

(7172, 785)

In [ ]:
df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,pixel10,pixel11,pixel12,pixel13,pixel14,pixel15,pixel16,pixel17,pixel18,pixel19,pixel20,pixel21,pixel22,pixel23,pixel24,pixel25,pixel26,pixel27,pixel28,pixel29,pixel30,pixel31,pixel32,pixel33,pixel34,pixel35,pixel36,pixel37,pixel38,pixel39,...,pixel745,pixel746,pixel747,pixel748,pixel749,pixel750,pixel751,pixel752,pixel753,pixel754,pixel755,pixel756,pixel757,pixel758,pixel759,pixel760,pixel761,pixel762,pixel763,pixel764,pixel765,pixel766,pixel767,pixel768,pixel769,pixel770,pixel771,pixel772,pixel773,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,3,107,118,127,134,139,143,146,150,153,156,158,160,163,165,159,166,168,170,170,171,171,171,172,171,171,170,170,169,111,121,129,135,141,144,148,151,154,157,160,...,205,206,206,207,207,206,206,204,205,204,203,202,142,151,160,172,196,188,188,190,135,96,86,77,77,79,176,205,207,207,207,207,207,207,206,206,206,204,203,202
1,6,155,157,156,156,156,157,156,158,158,157,158,156,154,154,153,152,151,149,149,148,147,146,144,142,143,138,92,108,158,159,159,159,160,160,160,160,160,160,160,...,100,78,120,157,168,107,99,121,133,97,95,120,135,116,95,79,69,86,139,173,200,185,175,198,124,118,94,140,133,84,69,149,128,87,94,163,175,103,135,149
2,2,187,188,188,187,187,186,187,188,187,186,185,185,185,184,184,184,181,181,179,179,179,178,178,109,52,66,77,83,188,189,189,188,188,189,188,188,188,188,187,...,203,204,203,201,200,200,199,198,196,195,194,193,198,166,132,114,89,74,79,77,74,78,132,188,210,209,206,205,204,203,202,201,200,199,198,199,198,195,194,195
3,2,211,211,212,212,211,210,211,210,210,211,209,207,208,207,206,203,202,201,200,198,197,195,192,197,171,51,52,54,212,213,215,215,212,212,213,212,212,211,211,...,247,242,233,231,230,229,227,225,223,221,220,216,58,51,49,50,57,60,17,15,18,17,19,1,159,255,237,239,237,236,235,234,233,231,230,226,225,222,229,163
4,13,164,167,170,172,176,179,180,184,185,186,188,189,189,190,191,189,190,190,187,190,192,193,191,191,192,192,194,194,166,169,172,174,177,180,182,185,186,187,190,...,90,77,88,117,123,127,129,134,145,152,156,179,105,106,105,104,104,104,175,199,178,152,136,130,136,150,118,92,85,76,92,105,105,108,133,163,157,163,164,179


In [ ]:
labels = df['label'].values
test_label = df_test['label'].values

In [ ]:
pixel = df.drop('label',axis=1).values
test_pixel = df_test.drop('label',axis=1).values

In [ ]:
count = 0
img = []
for data in pixel:
  ima = data.reshape(28,28)
  img.append(ima)
  
  


In [ ]:
test_img = []
for data in test_pixel:
  ima = data.reshape(28,28)
  test_img.append(ima)

In [ ]:
img_test_array = np.array(test_img).astype(float)

In [ ]:
img_array = np.array(img).astype(float)

In [ ]:
img_array.shape

(27455, 28, 28)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
training_img = np.expand_dims(img_array,axis=3)
testing_img = np.expand_dims(img_test_array,axis=3)

In [ ]:
training_img.shape

(27455, 28, 28, 1)

In [ ]:
train_imggen = ImageDataGenerator(rescale=1./255.,shear_range=0.2,zoom_range=0.2)
test_imggen = ImageDataGenerator(rescale=1./255.)

In [ ]:
train_datagen = train_imggen.flow(training_img,labels,batch_size=50)
#test_datagen = test_imggen()

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(26, activation='softmax')
])


In [ ]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history = model.fit_generator(
    train_datagen,
    epochs=10)

Epoch 1/10
550/550 [==============================] - 22s 41ms/step - loss: 1.2544 - accuracy: 0.6272
Epoch 2/10
550/550 [==============================] - 22s 41ms/step - loss: 0.2127 - accuracy: 0.9361
Epoch 3/10
550/550 [==============================] - 22s 40ms/step - loss: 0.0755 - accuracy: 0.9802
Epoch 4/10
550/550 [==============================] - 22s 40ms/step - loss: 0.0365 - accuracy: 0.9910
Epoch 5/10
550/550 [==============================] - 22s 40ms/step - loss: 0.0279 - accuracy: 0.9923
Epoch 6/10
550/550 [==============================] - 22s 40ms/step - loss: 0.0256 - accuracy: 0.9923
Epoch 7/10
550/550 [==============================] - 22s 41ms/step - loss: 0.0102 - accuracy: 0.9976
Epoch 8/10
550/550 [==============================] - 22s 41ms/step - loss: 0.0209 - accuracy: 0.9940
Epoch 9/10
550/550 [==============================] - 22s 41ms/step - loss: 0.0089 - accuracy: 0.9973
Epoch 10/10
550/550 [==============================] - 22s 40ms/step - loss: 0.009

In [ ]:
model.evaluate(testing_img,test_label)

225/225 [==============================] - 2s 8ms/step - loss: 65.5686 - accuracy: 0.9332


[65.56861114501953, 0.9332125186920166]

In [ ]:

from keras.models import load_model

model.save('sign_mnist.h5')